<a href="https://colab.research.google.com/github/AmrHassanKhalaf/Summer_Training_NLP/blob/main/Sec_10/Task_2/Arabic_Sentiment_Analysis_Sec_10_T_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df=pd.read_csv('/content/drive/MyDrive/Data/Arabic Sentiment Analysis.csv')

In [4]:
df.head()

,review_description,rating
0,شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...,-1
1,خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...,1
2,تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...,-1
3,فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...,1
4,سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...,-1


In [5]:
import re

def clean_arabic(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r"[\u0617-\u061A\u064B-\u0652]", "", text)
    text = re.sub(r"\u0640", "", text)
    text = re.sub(r"[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub(r"[0-9٠-٩]", " ", text)
    text = re.sub(r"[A-Za-z]", " ", text)
    text = re.sub(r"[^ء-ي\s]", " ", text)
    text = re.sub(r"(.)\1{2,}", r"\1", text)
    text = re.sub(r"(\b\w+\b)(\s+\1)+", r"\1", text)
    text = re.sub(r"\s+", " ", text).strip()

    return text


In [6]:
df["clean_review"] = df["review_description"].apply(clean_arabic)

print(df[["review_description", "clean_review"]].head(10))

                                  review_description  \
0  شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...   
1  خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...   
2  تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...   
3  فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...   
4  سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...   
5  قعد عشرين سنة يدور على سائق بس اما عن توصيل ال...   
6                                         احلئ تطبيق   
7                                      رائع واو مدهش   
8  مکو بس البحرین وعمان وغیرهه بس العراق مکو یعنی...   
9                    تطبيق جميل يستاهل الخمس نجوم👍👍👍   

                                        clean_review  
0  شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...  
1  خدمه الدفع عن طريق الكي نت توقفت عندي اصبح فقط...  
2  تطبيق غبي و جاري حذفه عاملين اكواد خصم و لما ن...  
3  فعلا تطبيق ممتاز بس لو في امكانيه يتيح لمستخدم...  
4     سيء جدا اسعار رسوم التوصيل لا تمت للواقع ب صله  
5  قعد عشرين سنه يدور علي سائق بس اما عن توصيل ال... 

In [7]:

df = df[df["clean_review"].str.strip() != ""].reset_index(drop=True)

print(df.shape)
print(df.head(5))


(30418, 3)
                                  review_description  rating  \
0  شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...      -1   
1  خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...       1   
2  تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...      -1   
3  فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...       1   
4  سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...      -1   

                                        clean_review  
0  شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...  
1  خدمه الدفع عن طريق الكي نت توقفت عندي اصبح فقط...  
2  تطبيق غبي و جاري حذفه عاملين اكواد خصم و لما ن...  
3  فعلا تطبيق ممتاز بس لو في امكانيه يتيح لمستخدم...  
4     سيء جدا اسعار رسوم التوصيل لا تمت للواقع ب صله  


In [8]:
df.drop(columns=["review_description"],inplace=True)

In [9]:
df.head()

,rating,clean_review
0,-1,شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...
1,1,خدمه الدفع عن طريق الكي نت توقفت عندي اصبح فقط...
2,-1,تطبيق غبي و جاري حذفه عاملين اكواد خصم و لما ن...
3,1,فعلا تطبيق ممتاز بس لو في امكانيه يتيح لمستخدم...
4,-1,سيء جدا اسعار رسوم التوصيل لا تمت للواقع ب صله


In [10]:
texts = df["clean_review"].astype(str).tolist()
label = df["rating"].replace({-1: 0, 1: 1}).tolist()

In [11]:
print(df.shape)

(30418, 2)


In [12]:
df = df.dropna(subset=["clean_review", "rating"])
X = df["clean_review"].astype(str).values
y = df["rating"].apply(lambda x: 1 if x > 0 else 0).values


In [26]:
X.shape

(30418,)

In [22]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# إعداد المعلمات
vocab_size = 32000   # عدد الكلمات القصوى في القاموس
max_len = 100        # طول الجملة بعد الـ padding

# تحويل النصوص لأرقام
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(X)

sequences = tokenizer.texts_to_sequences(X)
padded = pad_sequences(sequences, maxlen=max_len, padding="post", truncating="post")

# تقسيم البيانات Train/Test
X_train, X_test, y_train, y_test = train_test_split(padded, y, test_size=0.2, random_state=42)


In [27]:
X_train.shape

(24334, 100)

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout


embedding_dim = 200

model = Sequential([
   Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    SimpleRNN(64, return_sequences=False),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])



In [24]:
model.build(input_shape=(None, max_len))
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 100, 200)       │     6,400,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (None, 64)             │        16,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,425,409 (24.51 MB)

 Trainable params: 6,425,409 (24.51 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=64, epochs=5, validation_split=0.2)

Epoch 1/5
305/305 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.5742 - loss: 0.6823 - val_accuracy: 0.7684 - val_loss: 0.5223
Epoch 2/5
305/305 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.7996 - loss: 0.4819 - val_accuracy: 0.8110 - val_loss: 0.4375
Epoch 3/5
305/305 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.8678 - loss: 0.3491 - val_accuracy: 0.8208 - val_loss: 0.4347
Epoch 4/5
305/305 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9029 - loss: 0.2756 - val_accuracy: 0.8200 - val_loss: 0.4499
Epoch 5/5
305/305 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.9218 - loss: 0.2270 - val_accuracy: 0.8192 - val_loss: 0.4732


In [28]:
model.save('sentiment_model.h5')

In [32]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=64, epochs=8, validation_split=0.2)

Epoch 1/8
305/305 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.9275 - loss: 0.2156 - val_accuracy: 0.7865 - val_loss: 0.5615
Epoch 2/8
305/305 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.9404 - loss: 0.1752 - val_accuracy: 0.8079 - val_loss: 0.5853
Epoch 3/8
305/305 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9264 - loss: 0.2095 - val_accuracy: 0.7510 - val_loss: 0.5891
Epoch 4/8
305/305 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.8396 - loss: 0.4001 - val_accuracy: 0.7473 - val_loss: 0.5580
Epoch 5/8
305/305 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.6650 - loss: 0.6225 - val_accuracy: 0.6392 - val_loss: 0.6522
Epoch 6/8
305/305 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7021 - loss: 0.6065 - val_accuracy: 0.6394 - val_loss: 0.6604
Epoch 7/8
305/305 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.7039 - loss: 0.6049 - val_accuracy: 0.6402 - val_loss: 0.6662
Epoch 8/8
305/305 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.7091 - loss: 0.6018 - val_accuracy: 0

In [ ]:
    # dropout_rate = 0.3
    # Dropout(dropout_rate),

In [29]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")

191/191 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8258 - loss: 0.4666
Test Accuracy: 0.8263


In [30]:
sample = ["التطبيق ممتاز وسهل جدا"]
sample_vec = tokenizer.texts_to_sequences(sample)
sample_vec = pad_sequences(sample_vec, maxlen=max_len, padding="post", truncating="post")
prediction = model.predict(sample_vec)
print("إيجابي" if prediction[0][0] > 0.5 else "سلبي")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 411ms/step
إيجابي


In [31]:
sample = ["التطبيق سيء جدا ومليان مشاكل"]
sample_vec = tokenizer.texts_to_sequences(sample)
sample_vec = pad_sequences(sample_vec, maxlen=max_len, padding="post", truncating="post")
prediction = model.predict(sample_vec)
print("إيجابي" if prediction[0][0] > 0.5 else "سلبي")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
سلبي


In [ ]:
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout




# Tokenization
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts)
X = tokenizer.texts_to_sequences(texts)
X = pad_sequences(X, maxlen=20)

y = np.array(label)

# بناء الموديل RNN
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=20))
model.add(SimpleRNN(128, activation="tanh", return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# تدريب الموديل
model.fit(X, y, epochs=10, batch_size=2, verbose=1)


Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


 1605/15209 ━━━━━━━━━━━━━━━━━━━━ 3:03 13ms/step - accuracy: 0.6035 - loss: 0.6806